# Setup

In [29]:
import numpy as np
import pandas as pd
from scipy import special
from copy import copy
import math
import seaborn as sns
import importlib
import matplotlib.pyplot as plt
import warnings
from scipy.stats import norm
from tqdm import tqdm
warnings.filterwarnings("ignore")

import sklearn.linear_model as lm

import dml_test
importlib.reload(dml_test)

sns.set(font_scale=2,style='whitegrid')

STAR_High_School = pd.read_spss('PROJECTSTAR/STAR_High_Schools.sav')
STAR_K3_School = pd.read_spss('PROJECTSTAR/STAR_K-3_Schools.sav').set_index('schid')
STAR_Students = pd.read_spss('PROJECTSTAR/STAR_Students.sav').set_index('stdntid')
Comparison_Students = pd.read_spss('PROJECTSTAR/Comparison_Students.sav').set_index('stdntid')

# pre-treatment covariates
gk_cols = list(filter(lambda x: 'gk' in x, STAR_Students.columns))
g1_cols = list(filter(lambda x: 'g1' in x, STAR_Students.columns))
g2_cols = list(filter(lambda x: 'g2' in x, STAR_Students.columns))
g3_cols = list(filter(lambda x: 'g3' in x, STAR_Students.columns))
g_cols = gk_cols+g1_cols+g2_cols+g3_cols

personal_cols = ['gender','race','birthyear']

cols_cond = ['surban',
            'tgen',
            'trace',
            'thighdegree',
            'tcareer',
            'tyears',
            'classsize',
            'freelunch']

class_sizes = ['g1classsize',
             'g2classsize']

g3scores = ['g3treadss',
            'g3tmathss',
            'g3tlangss',
            'g3socialsciss']

g_cols_cond = list(filter(lambda s: np.sum(list(map(lambda x: x in s,cols_cond)))>0,g_cols))
df_exp = STAR_Students[personal_cols]#+class_sizes]
df_exp['Sample'] = 1
df_exp['g3avgscore'] = STAR_Students[g3scores].mean(axis=1)
df_exp['g3smallclass'] = (STAR_Students['g3classsize']<=17).astype(int)

df_obs = Comparison_Students[personal_cols]#+class_sizes]
df_obs['Sample'] = 0
df_obs['g3avgscore'] = Comparison_Students[g3scores].mean(axis=1)
df_obs['g3smallclass'] = (Comparison_Students['g3classsize']<=17).astype(int)

df = df_exp.append(df_obs)
df_no_na = df.dropna()

# df_no_na['birthmonth'].replace({'JANUARY':1.0,
#                                'FEBRUARY':2.0,
#                                'MARCH':3.0,
#                                'APRIL':4.0,
#                                 'ARPIL':4.0,
#                                'MAY':5.0,
#                                'JUNE':6.0,
#                                'JULY':7.0,
#                                'AUGUST':8.0,
#                                'SEPTEMBER':9.0,
#                                'OCTOBER':10.0,
#                                'NOVEMBER':11.0,
#                                 'DECEMBER':12.0},inplace=True)
df_no_na_dummified = pd.get_dummies(df_no_na,columns=['gender','race'],drop_first=True)

In [24]:
df_exp.to_csv('Project_STAR_exp.csv')
df_obs.to_csv('Project_STAR_obs.csv')

In [25]:
df_no_na_dummified

birthyear  Sample  g3avgscore  g3smallclass  gender_MALE  race_BLACK  \
stdntid                                                                         
10000.0     1979.0       1      597.00             0            1           0   
10002.0     1979.0       1      576.50             0            0           1   
10003.0     1980.0       1      616.50             0            1           0   
10004.0     1980.0       1      619.00             0            0           1   
10005.0     1980.0       1      597.50             0            1           1   
...            ...     ...         ...           ...          ...         ...   
31757.0     1979.0       0      666.50             0            1           0   
31767.0     1980.0       0      674.25             0            1           0   
31774.0     1980.0       0      669.25             0            0           0   
31776.0     1980.0       0      635.00             0            0           0   
31777.0     1980.0       0      604.25             0            1           1   

         race_HISPANIC  race_OTHER  race_WHITE  
stdntid                                         
10000.0              0           0           1  
10002.0              0           0           0  
10003.0              0           0           1  
10004.0              0           0           0  
10005.0              0           0           0  
...                ...         ...         ...  
31757.0              0           0           1  
31767.0              0           0           1  
31774.0              0           0           1  
31776.0              0           0           1  
31777.0              0           0           0  

[7049 rows x 9 columns]

# DML Test

In [26]:
N = df_no_na_dummified.shape[0]
df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df = dml_test.fit('g3avgscore','g3smallclass', 
                                                            df_exp=df_no_na_dummified.loc[df_no_na_dummified['Sample']==1].drop(columns=['Sample']), 
                                                            df_obs=df_no_na_dummified.loc[df_no_na_dummified['Sample']==0].drop(columns=['Sample']),
                                                            n_splits = 100)
psi = dml_test.Psi(df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df, 'g3avgscore', 'g3smallclass')
p_val = dml_test.dml_pval( psi)

In [27]:
print('p(1) = %.6f, p(0) = %.6f'%(p_val[0],p_val[1]))

p(1) = 0.005391, p(0) = 0.000000


In [28]:
np.mean(psi,axis=0)

array([18.87696531, 10.87456877])

In [32]:
fig = plt.figure(figsize=(10,5))
N = psi.shape[0]
thetas = []
for repeat in range(1000):
    idx = list(np.random.randint(0,N,size=N))
    psi_bs = psi[idx,:]
    thetas += [np.nanmean(psi_bs, axis = 0)]
thetas = np.array(thetas)
sns.kdeplot(thetas[:,0],fill=True)
sns.kdeplot(thetas[:,1],fill=True)
# ax = fig.add_subplot(1, 1, 1)
# ax.set_facecolor('#FAFAFA')
# fig.patch.set_facecolor('#FAFAFA')
plt.legend([r'$\theta(1)$',r'$\theta(0)$'])
plt.axvline(0,c='black',ls='--')
# plt.title('p(1) = %.3f, p(0) = %.3f'%(p_val[0],p_val[1]))
fig.savefig('psi_STAR.png')

# Sensitivity Analysis

$Y^{debias} = Y_i  - q(T_i,X_i) P(1-T_i | X_i)$

In [12]:
propensity_model = lm.LogisticRegressionCV().fit(df_no_na_dummified.drop(columns=['g3smallclass','g3avgscore']).loc[df_no_na_dummified['Sample']==0],
                                                 df_no_na_dummified.loc[df_no_na_dummified['Sample']==0]['g3smallclass'])
pi = propensity_model.predict_proba(df_no_na_dummified.drop(columns=['g3smallclass','g3avgscore']).loc[df_no_na_dummified['Sample']==0])
w = df_no_na_dummified.loc[df_no_na_dummified['Sample']==0]['g3smallclass'] * pi[:,0] + (1-df_no_na_dummified.loc[df_no_na_dummified['Sample']==0]['g3smallclass']) * pi[:,1]

def q(a,T):
    return a*(2*T - 1)

n_alpha = 50
p_array = []
for a in tqdm(np.linspace(0, 50, num= n_alpha)):
    df_no_na_dummified_copy = df_no_na_dummified.copy(deep=True)
    df_no_na_dummified_copy.loc[df_no_na_dummified_copy['Sample']==0,'g3avgscore'] = df_no_na_dummified_copy.loc[df_no_na_dummified_copy['Sample']==0]['g3avgscore'] - q(a,df_no_na_dummified_copy.loc[df_no_na_dummified_copy['Sample']==0]['g3smallclass']) * w
    df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df = dml_test.fit('g3avgscore','g3smallclass', 
                                                            df_exp=df_no_na_dummified_copy.loc[df_no_na_dummified_copy['Sample']==1].drop(columns=['Sample']), 
                                                            df_obs=df_no_na_dummified_copy.loc[df_no_na_dummified_copy['Sample']==0].drop(columns=['Sample']),
                                                            n_splits = 50)
    psi = dml_test.Psi(df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df, 'g3avgscore', 'g3smallclass')
    p_val = dml_test.dml_pval(psi)
    p_array.append( p_val )
    


100%|██████████| 50/50 [58:41<00:00, 70.43s/it]


In [13]:
pd.DataFrame(np.array(p_array),columns=['p(1)','p(0)']).to_csv('star_pvals.csv')

In [14]:
sns.set_style("whitegrid")
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('#FAFAFA')
fig.patch.set_facecolor('#FAFAFA')
plt.plot(np.linspace(0, 150,num = n_alpha),np.array(p_array)[:,0],lw = 2.5, c = '#EB811B')
# plt.plot(np.linspace(0, 150,num = n_alpha),np.array(p_array)[:,1],lw = 2.5)
plt.axhline(0.05, c = '#48858C', ls = '--')
# plt.axvline(df_exp.loc[df_exp['g3smallclass']==1,'g3avgscore'].mean()-df_exp.loc[df_exp['g3smallclass']==0,'g3avgscore'].mean(),lw=2.5)
# plt.axvline(np.mean(te,axis=0)[0]- np.mean(te,axis=0)[1],c='black',lw=2.5)
# plt.axvline(np.std(te,axis=0)[0]- np.std(te,axis=0)[1],c='black',lw=2.5)
plt.ylabel('p(1)')
plt.xlabel(r'$\alpha$')
plt.title(r'selection bias: $ q(X,T) = \alpha (2T - 1)$')
plt.savefig('sensitivity_test_STAR.png')

In [18]:
df_no_na_dummified['g3avgscore'].min()

486.0

## Leave One Out (Cluster) 

In [ ]:
p_array = np.load('all_pvals.npy')

In [ ]:
np.array(p_array).shape

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1, 1, 1)
ax.set_facecolor('#FAFAFA')
fig.patch.set_facecolor('#FAFAFA')
plt.plot(np.arange(0, 350, step = 3)[1:100] / 400, np.array(p_array)[1:100, 0],lw = 2.5, c = '#EB811B')
plt.axhline(0.05, c = '#48858C', ls = '--')
plt.ylabel('P Value')
plt.xlabel(r'$\alpha$')
# plt.title(r'Sensitivity analysis for STAR DataSelection Bias: $\alpha(2T - 1)$')
plt.savefig('sensitivity_test_STAR.png')

# ATE Estimation

In [7]:
te = dml_test.Lambda(df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df, 'g3avgscore','g3smallclass')

In [12]:
np.mean(te,axis=0)[0]- np.mean(te,axis=0)[1], (np.std(te,axis=0)[0] + np.std(te,axis=0)[1])/np.sqrt(df.shape[0])

(7.385971623391242, 1.4327841167099227)

In [9]:
#ATE Exp estimate
df_exp.loc[df_exp['g3smallclass']==1,'g3avgscore'].mean()-df_exp.loc[df_exp['g3smallclass']==0,'g3avgscore'].mean()

7.233299739829135

In [10]:
df_exp.loc[df_exp['g3smallclass']==1,'g3avgscore'].std()/np.sqrt(df_exp.loc[df_exp['g3smallclass']==1,'g3avgscore'].shape[0])
+df_exp.loc[df_exp['g3smallclass']==0,'g3avgscore'].std()/np.sqrt(df_exp.loc[df_exp['g3smallclass']==0,'g3avgscore'].shape[0])

0.36014905111987944

In [ ]:
def IPW(df, outcome, treatment):
    S = df['S']
    
    n = len(S)

    p = np.mean(S)
    
    T = df[treatment]
    Y = df[outcome]
    
    return np.sum(S * T * Y) / np.sum(S * T) - np.sum(S * (1 - T) * Y) / np.sum(S * (1 - T))

def Lambda(df_mu, df_nu, df_p, df_pi_exp, df_pi_obs, df, outcome, treatment):
    """Construct the estimator Lambda for the observational ATE."""
    mu0 = df_nu.iloc[:, 1].values # mu(X, 0, 0) in the current notation
    mu1 = df_nu.iloc[:, 0].values # mu(X, 1, 0) in the current notation
    
    S = df['S'].values
    T = df[treatment].values
    Y = df[outcome].values
    
    p = df_p.values[:,0]
    pi_exp = df_pi_exp.values[:,0]
    
    lambda0 = ( S * (1-T) * Y ) / ( p * ( 1 - pi_exp ) ) + ( mu0 * ( 1 - ( S*(1-T)/( p * ( 1 - pi_exp ) ) ) ) )
    lambda1 = ( S * (T) * Y ) / ( p * ( pi_exp ) ) + ( mu0 * ( 1 - ( S*(T)/( p * ( pi_exp ) ) ) ) )
    
    return np.vstack((lambda1, lambda0)).T